<a href="https://colab.research.google.com/github/jong9810/TensorFlow-2.0/blob/main/8_2_GRU_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8-2. GRU Example : 삼성전자 주가 예측

## 머신러닝 프로그램 개발 프로세스
1. 데이터 로드 및 분포 확인 (그래프 그려보기)
1. 데이터 전처리 (Outlier, Missing value, 정규화, 표준화, One-Hot Encoding, feature / label 데이터 정의 등)
1. 데이터 생성 (feature / label 데이터를 시계열 데이터 형식으로 변환, window size, horizontal factor 지정)
1. 데이터 분리 및 모델 구축
1. 데이터 예측 및 모델 평가 (정확도, 손실함수 값, 오차 등 그래프로 그려보기)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

tf.__version__

In [ ]:
# 구글 드라이브를 구글 코랩에 마운트 시킴
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# 1. 데이터 로드 및 분포 확인
raw_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset/005930.KS_3MA_5MA.csv')
raw_df.head()

In [ ]:
plt.figure(figsize=(7,4))
plt.plot(raw_df['Adj Close'], label='Adj Close', color='blue')

plt.title('SAMSUNG ELECTRONIC STOCK PRICE')
plt.xlabel('period (day)')
plt.ylabel('price (won)')
plt.legend(loc='best')
plt.grid()
plt.show()

In [ ]:
# 2. 데이터 전처리
raw_df.describe()

In [ ]:
# 모든 열에 NaN 값이 몇 개 있는지 출력
raw_df.isnull().sum()

In [ ]:
raw_df.loc[raw_df['Open'].isna()]  # raw_df의 데이터 중 'Open' 열의 값이 NaN인 행을 모두 출력함

# raw_df의 데이터 중 'Volume' 열의 값이 0인 셀을 찾아서 NaN 값으로 바꿈
# => describe() 함수에서 Volume 열의 최소값이 0이었기 때문에 replace()함수로 대체해줌
raw_df['Volume'] = raw_df['Volume'].replace(0, np.nan)

raw_df.isnull().sum()

In [ ]:
# 데이터의 0 값 모두 NaN으로 잘 바뀌었는지 확인
for col in raw_df.columns:
    missing_rows = raw_df.loc[raw_df[col]==0].shape[0]
    print(col + ' : ' + str(missing_rows))

In [ ]:
raw_df = raw_df.dropna() # NaN 값이 있는 행의 데이터를 모두 삭제해줌
raw_df.isnull().sum()

In [ ]:
# 데이터 생성
from sklearn.preprocessing import MinMaxScaler # 데이터를 정규화 해주는 함수

In [ ]:
scaler = MinMaxScaler()

scale_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', '3MA', '5MA']
scaled_df = scaler.fit_transform(raw_df[scale_cols])
# print(type(scaled_df))
# print(scaled_df)

In [ ]:
scaled_df = pd.DataFrame(scaled_df, columns=scale_cols)
# print(scaled_df, type(scaled_df))

In [ ]:
feature_cols = ['3MA', '5MA', 'Adj Close']
label_cols = ['Adj Close']

feature_df = pd.DataFrame(scaled_df, columns=feature_cols)
label_df = pd.DataFrame(scaled_df, columns=label_cols)

# print(feature_df)
# print(label_df)

feature_np = feature_df.to_numpy()
label_np = label_df.to_numpy()

# print(type(feature_np))
# print(type(label_np))

In [ ]:
def make_sequence_dataset(feature, label, w, h):
    feature_list = []
    label_list = []

    for i in range(len(feature) - w - h + 1):
        feature_list.append(feature[i:i+w])
        label_list.append(label[i+w+h-1])

    return np.array(feature_list), np.array(label_list)

In [ ]:
w = 40 # window size
h = 1 # horizontal factor

X, Y = make_sequence_dataset(feature_np, label_np, w, h)

In [ ]:
# 4, 데이터 분리 및 모델 구축
split = -200
x_train = X[:split]
y_train = Y[:split]

x_test = X[split:]
y_test = Y[split:]

# print(x_train.shape, y_train.shape)
# print(x_test.shape, y_test.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU

In [ ]:
model = Sequential()

model.add(GRU(256, activation='tanh', input_shape=x_train[0].shape))
model.add(Dense(1, activation='linear'))
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model.compile(loss='mse', metrics=['mae'], optimizer='adam')

early_stop = EarlyStopping(monitor='val_loss', patience=5)

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=16, callbacks=[early_stop])

In [ ]:
pred = model.predict(x_test)

plt.figure(figsize=(12,6))
plt.title('3MA + 5MA + Adj Close, window_size=40')
plt.xlabel('period')
plt.ylabel('adj close')
plt.plot(y_test, label='actual')
plt.plot(pred, label='prediction')
plt.legend(loc='best')
plt.grid()

plt.show

In [ ]:
model.evaluate(x_test, y_test)

### LSTM Vs. GRU
- LSTM과 GRU 모두 시계열 데이터(ex. 주식 데이터)를 분석, 예측하기에 적합한 구조라는 것을 알 수 있었음
- 다만 LSTM은 업데이트하는 가중치, 바이어스가 GRU보다 많아서 계산량은 더 많고 정확도는 좀 더 높다는 것도 알 수 있었다.
- 따라서 분석하고자 하는 문제와 데이터, 도메인 등에 따라 GRU를 사용할지 LSTM을 사용할지를 적절히 선택하는 것이 중요하다.